# Biopython: a minimal sampler

**NOTE**: This notebook assumes that you have studied the notebook on Object Oriented Programming and done the exercises.

For these materials to work, you must have Biopython installed on your system, that is, you should be able to run the Python instructions ```import Bio``` successfully.

Biopython is already installed on the Jupyter Hub. If it is not installed on your system, please check out the [packages](https://biopython.org/wiki/Packages) page, or the general [installation instructions](https://biopython.org/wiki/Download).

## Introduction, scope and limitations

[Biopython](https://biopython.org/) is a large and heterogenous Python library that supports several common bioinformatics tasks, including sequence I/O, access to the main online databases, sequence alignment, motif construction and search, the analysis of phenotypic data, and some machine learning algorithms. It is the alternative to [BioPerl](https://bioperl.org/) in the Python world.

A detailed treatment of Biopython is beyond the scope of this module. This notebook highlights the points of contact between the design and functioning of Biopython and the concepts we have covered so far; the aim is to give you a few entry points and to convince you that you now have all the knowledge and tools required to learn the relevant parts of the library and apply them to your specific problems, when need arises.

## General structure and entry points

Biopython is an object oriented library. The [class diagram](https://biopython.org/DIST/docs/api/class-tree.html) is extensive. It describes the inheritance relation between all classes in the library. However it is best used as a reference, and is not a good access point for learning the library. The reason is that, as we will see, the most important relation between objects for an understanding of Biopython is the *has-a* relationship (e.g., a Sequence Record object has an attribute that is a Sequence object) rather than the *is-a* relationship of inheritance. The [tutorial](http://biopython.org/DIST/docs/tutorial/Tutorial.html) is a far more accessible entry point, and I recommend that you have a look. This sampler discusses a few examples from the tutorial, without any pretense at completeness.

## The Seq object

In [ ]:
from Bio.Seq import Seq

The [Seq](https://biopython.org/docs/1.75/api/Bio.Seq.html) class is maybe the single most important class; it is immutable (though a mutable version exists), and it is basically a string with an alphabet, and a few added functions.

In [ ]:
my_seq=Seq("ATCGCCTAACGATGA")

In [ ]:
# you can slice it 
my_seq[4:7]

In [ ]:
# you can loop over it
for (pos, base) in enumerate(my_seq):
    print (pos, base)

In [ ]:
# you will find some old friends
my_seq.count("A")

The ```Seq``` object, obviously, also offers some specific methods:

In [ ]:
# complement of the sequence, does not reverse the string
my_seq.complement()

In [ ]:
# complement of the sequence, reverses the string
my_seq.reverse_complement()

In [ ]:
# from DNA to RNA
print(my_seq.transcribe())

In [ ]:
# Translate works both on DNA and or RNA sequences.
# There are two stop codons in this sequence, TAA and TGA
my_seq.translate()

As the output of ```.translate()``` showns, the ```Seq``` object is also suitable for aminoacid sequences:

In [ ]:
aa_seq=Seq("MEEPQ---SDPSVEPPLSQ--ETFSDLWK---LLPE")
aa_seq

In [ ]:
# remove gaps
aa_seq.ungap()

Some ```Seq``` methods may not make sense all the time:

In [ ]:
aa_seq.complement() # ????? meaningless output

```Seq``` supports most applicable string operations, but explicit conversion to/from ```str``` is obviously possible: 

In [ ]:
str_seq=str(aa_seq)
str_seq

In [ ]:
Seq(str_seq)

## The SeqIO package

In [ ]:
from Bio import SeqIO

The ```SeqIO``` package provides parsers for lots of file formats. Technically these parsers are *iterables* that load one sequence at a time in the form of a [SeqRecord](https://biopython.org/docs/1.75/api/Bio.SeqRecord.html) object. A ```SeqRecord``` wraps a ```Seq``` object with the actual sequence together with any additional information such as accession numbers, description, annotation, etc.


### Reading FASTA files

As an example, here is how we obtain a parser object for a FASTA file:

In [ ]:
parser=SeqIO.parse("data/P04637.fas", "fasta")
print (type(parser)) # as we see, parse returns a specific iterator for the Fasta format

If the file has more than one sequence in it, we can loop over the ```parser``` object much as we would over a file handle (see the examples [here](http://biopython.org/DIST/docs/tutorial/Tutorial.html#sec54)), and it will return sequence records one at a time. However, our file only contains one sequence, so we can simply use the ```next``` keyword to read it:

In [ ]:
prot_rcd=next(parser)
print(type(prot_rcd))

as said, we obtain a [SeqRecord](https://biopython.org/docs/1.75/api/Bio.SeqRecord.html) object. This provides the actual sequence object in the ```.seq``` attribute, together with the additional information included in the file: 

In [ ]:
dir (prot_rcd)

In [ ]:
prot_rcd.name

In [ ]:
prot_rcd.description

In [ ]:
prot_rcd.annotations # no annotations - an empty dict

In [ ]:
# this is the Seq object with the actual sequence...
type(prot_rcd.seq)

In [ ]:
# ...so here is the sequence
print(prot_rcd.seq)

### Reading GenBank files

The same workflow can be used to read, for instance, [GenBank](https://www.ncbi.nlm.nih.gov/genbank/) data. Here I have downloaded the [genetic sequence](https://www.ncbi.nlm.nih.gov/nuccore/NG_017013) that encodes for the P53 protein, in the GenBank-specific ```.gb``` format. As we can see by inspecting it with an editor, the actual file (in *data/NG_017013.gb*) contains all the rich information displayed on the webpage. The workflow, however, is largely the same as above: we first obtain the parser object

In [ ]:
gb_parser=SeqIO.parse("data/NG_017013.gb", "gb")
print (type(gb_parser)) # as we see, in this case we get a GenBankIterator

and we then read the first (and only) record out of it

In [ ]:
gene_rcd=next(gb_parser)
print (type(gene_rcd)) # this is the familiar SeqRecord

This time, the ```SeqRecord``` object carries some more information:

In [ ]:
gene_rcd.name

In [ ]:
gene_rcd.description

In [ ]:
gene_rcd.annotations

The ```.seq``` attribute is still where the sequence is found:

In [ ]:
print(type(gene_rcd.seq))

In [ ]:
len(gene_rcd.seq)

In [ ]:
gene_rcd.seq

## The SearchIO package

In [ ]:
from Bio import SearchIO

The [SearchIO](https://biopython.org/docs/1.76/api/Bio.SearchIO.html) package contains the Biopython interface for sequence search program outputs. From the documentation:

"SearchIO parses a search output file’s contents into a hierarchy of four nested objects: QueryResult, Hit, HSP, and HSPFragment. Each of them models a part of the search output file:

* ```QueryResult``` represents a search query. This is the main object returned by the input functions and it contains all other objects.

* ```Hit``` represents a database hit,

* ```HSP``` represents high-scoring alignment region(s) in the hit,

* ```HSPFragment``` represents a contiguous alignment within the HSP"



As an example, I dowloaded the output of a BLAST search of the P53 sequence against the *nr* database to a file, in *xml* format. The results can be read with the ```SearchIO.read``` function, that returns a ```QueryResult``` object.

In [ ]:
blast_qresult = SearchIO.read("data/P53-BLAST-out.xml", "blast-xml")
print(type(blast_qresult))

The ```QueryResult``` object contains all the hits; the ```len``` function returns the number of hits

In [ ]:
len(blast_qresult)

In [ ]:
# we can slice it
print(blast_qresult[:5])

By indexing the ```QueryObject``` we access the individual ```Hit``` objects

In [ ]:
top_hit=blast_qresult[0]
print (type(top_hit))

As can be seen, the ```Hit``` object contains a wealth of information about the search hit:

In [ ]:
dir (top_hit)

In [ ]:
# accession code of the hit
top_hit.accession

In [ ]:
# description of the hit
top_hit.description

The most interesting part is however the ```.hsps``` attribute, that is a list of the High-Scoring Pairs (HSPs) of regions in the hit.

In [ ]:
# here we only have one high scoring pair
top_hit.hsps

The HSP is encoded as a ```HSP``` object. The ```HSP``` object contains a wealth of information about the aligned regions, including evalue, bitscore and range of the match

In [ ]:
hsp=top_hit.hsps[0]
print(type(hsp))

In [ ]:
print (hsp)

In [ ]:
dir(hsp)

Further down the hierarchy, the ```.fragments``` field of the ```HSP``` object contains the fragments, encoded as ```HSPFragment``` objects

In [ ]:
hsp.fragments

# Conclusions

This notebook explores only a small fraction of the functionality offered by Biopython. From this quick overview, a few facts should be clear:
* Biopython constructs follow general Python coding practices and generally behave the way you would expect them to do;
* Biopython objects are interoperable with standard Python data types;
* the representation offered by Biopython is both flexible and powerful;
* unavoidably, there is a certain overhead involved in using it;
* and finally, there is a learning curve, but given a general knowledge of the language, it isn't a steep one.

As for all libraries, the benefit of using it should be evaluated on a case-by-case basis. Whether for instance you are better off using the "blast-xml" parser or writing some simple code to parse the "csv" output of a BLAST search largely depends on what information you need and what you want to do with it.

In my opinion, it is certainly worth knowing in general what the library has to offer. There may not, however, be much point in trying to learn the function of all of its classes for the sake of it - that time is better spent in improving one's general programming proficiency. One should probably learn how to use the appropriate parts of Biopython in a task-oriented way, as required by specific problems. 